In [5]:
import requests
import time
import pandas as pd

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
BASE_URL = "https://ows.goszakup.gov.kz"
API_ENDPOINT = "/v3/plans"
parquet_file = "D:/qbs/goszakup_plans_customer_list_data.parquet"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}  # Начинаем с первой страницы
all_data = []
seen_pids = set()
next_page = f"{API_ENDPOINT}?limit=500"  # Первоначальный запрос

while next_page:
    try:
        response = requests.get(f"{BASE_URL}{next_page}", headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Загрузка завершена, новых данных нет.")
            break

        # Фильтрация дубликатов
        new_items = [item for item in items if item["pid"] not in seen_pids]
        if not new_items:
            print("API вернуло дубликаты, остановка загрузки.")
            break

        all_data.extend(new_items)
        seen_pids.update(item["pid"] for item in new_items)

        print(f"Загружено {len(all_data)} записей...")

        # Получаем `next_page` из API
        next_page = data.get("next_page")
        if not next_page:
            print("Достигнута последняя страница, завершаем загрузку.")
            break

        # Промежуточное сохранение каждые 1000 записей
        if len(all_data) % 1000 < 500:
            pd.DataFrame(all_data).to_parquet(parquet_file, index=False, engine="pyarrow")
            print("Промежуточное сохранение...")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# Финальное сохранение
df = pd.DataFrame(all_data)
df.to_parquet(parquet_file, index=False, engine="pyarrow")

print(f"Данные сохранены в {parquet_file}")

Загружено 500 записей...
Загружено 1000 записей...
Промежуточное сохранение...
Загружено 1500 записей...
Загружено 1933 записей...
Загружено 2416 записей...
Промежуточное сохранение...
Загружено 2785 записей...
API вернуло дубликаты, остановка загрузки.
Данные сохранены в D:/qbs/goszakup_plans_customer_list_data.parquet


In [6]:
import numpy as np
import pandas as pd

df = pd.read_parquet("D:/qbs/goszakup_plans_customer_list_data.parquet")
df.head()

,pid,bin,name_ru,name_kz,doc_count
0,171777,161240020739,"Товарищество с ограниченной ответственностью ""...","""XAMID 001"" жауапкершілігі шектеулі серіктестігі",0
1,241268,010440001806,"ТОО ""ЭНКО-01""","""ЭНКО-01"" ЖШС",0
2,54126,031040007024,"РГУ ""Войсковая часть 01098"" Министерства оборо...",Қазақстан Республикасы Қорғаныс министрлігінің...,516
3,52017,020440006786,"РГУ ""Войсковая часть 01731"" Министерства оборо...",Қазақстан Республикасы Қорғаныс министрлігінің...,2915
4,105139,990240008066,"Республиканское государственное учреждение ""Во...",Қазақстан Республикасы Қорғаныс министрлігінің...,85


In [7]:
import numpy as np
df = df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

# Check for duplicates
duplicates = df.duplicated()
print(duplicates)

duplicates_count = df.duplicated().sum()
print(f'Количество дубликатов: {duplicates_count}')

0       False
1       False
2       False
3       False
4       False
        ...  
2780    False
2781    False
2782    False
2783    False
2784    False
Length: 2785, dtype: bool
Количество дубликатов: 0


C:\Users\alena\AppData\Local\Temp\ipykernel_17444\1402623328.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)
